In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

BASE_MODEL = "Qwen/Qwen3-1.7B"
LORA_PATH = "./qwen_nepali_budget_lora"

# -----------------------
# Load tokenizer
# -----------------------
tokenizer = AutoTokenizer.from_pretrained(LORA_PATH, trust_remote_code=True)

# -----------------------
# Load base model
# -----------------------
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# -----------------------
# Load LoRA adapters
# -----------------------
model = PeftModel.from_pretrained(model, LORA_PATH)



Loading weights: 100%|██████████| 311/311 [00:08<00:00, 34.56it/s, Materializing param=model.norm.weight]                              
The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


In [ ]:
test_samples = [
    {
        "instruction": "वैदेशिक सहायताको प्रभावकारिता कसरी मापन गरिन्छ?",
        "input": "",
        "response": "यथार्थ जानकारीका लागि आधिकारिक बजेट दस्तावेज हेर्नु उपयुक्त हुन्छ।"
    },
    {
        "instruction": "चालु आर्थिक वर्षको वैदेशिक मुद्रा सञ्चितिको अवस्था कस्तो छ?",
        "input": "",
        "response": "उक्त जानकारी उपलब्ध छैन।"
    },
    {
        "instruction": "दिइएको विवरणका आधारमा नवीकरणीय ऊर्जाका बारेमा लेख्नुहोस्।",
        "input": "ऊर्जा संकट समाधानका लागि सरकारले नवीकरणीय ऊर्जामा लगानी बढाउने नीति लिएको छ। सौर्य ऊर्जा र वायु ऊर्जा उत्पादन गर्ने उद्योगलाई कर छुटको व्यवस्था गरिएको छ। साथै, विद्युत खपत बढाउन विद्युतीय सवारी साधनको प्रयोगलाई प्रोत्साहन गरिनेछ।",
        "response": "दिइएको विवरण अनुसार नवीकरणीय ऊर्जाका लागि सौर्य र वायु ऊर्जा उत्पादन गर्ने उद्योगलाई कर छुट दिने र विद्युतीय सवारी साधनको प्रयोगलाई प्रोत्साहन गर्ने नीति लिइएको छ।"
    },
    # Add more examples as needed
]

# -----------------------
# Evaluation loop
# -----------------------
for i, sample in enumerate(test_samples):
    instruction = sample['instruction']
    context = sample['input']
    
    if context:
        prompt = f"### Instruction:\n{instruction}\n\n### Context:\n{context}\n\n### Response:\n"
    else:
        prompt = f"### Instruction:\n{instruction}\n\n### Response:\n"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        output = model.generate(
    **inputs,
    max_new_tokens=150,
    temperature=0.0,
    do_sample=False     
)
    
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print(answer)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Instruction:
वैदेशिक सहायताको प्रभावकारिता कसरी मापन गरिन्छ?

### Response:
वैदेशिक सहायताको प्रभावकारिता कसरी मापन गरिन्छ भन्ने विषयको जानकारी उपलब्ध नभएको छ।
### Instruction:
चालु आर्थिक वर्षको वैदेशिक मुद्रा सञ्चितिको अवस्था कस्तो छ?

### Response:
सरकारले वैदेशिक मुद्रा सञ्चितिको अवस्था निर्धारण गर्न आवश्यक छ। यसको विवरण विवरण विवरण गरिन्छ।
### Instruction:
दिइएको विवरणका आधारमा नवीकरणीय ऊर्जाका बारेमा लेख्नुहोस्।

### Context:
ऊर्जा संकट समाधानका लागि सरकारले नवीकरणीय ऊर्जामा लगानी बढाउने नीति लिएको छ। सौर्य ऊर्जा र वायु ऊर्जा उत्पादन गर्ने उद्योगलाई कर छुटको व्यवस्था गरिएको छ। साथै, विद्युत खपत बढाउन विद्युतीय सवारी साधनको प्रयोगलाई प्रोत्साहन गरिनेछ।

### Response:
नवीकरणीय ऊर्जा विकासका लागि सरकारले नीति लिएको छ। विद्युत खपत बढाउन विद्युतीय सवारी साधनको प्रयोगलाई प्रोत्साहन गरिनेछ। सौर ऊर्जा र वायु ऊर्जा उत्पादन गर्ने �


In [11]:
answer

'### Instruction:\nदिइएको विवरणका आधारमा नवीकरणीय ऊर्जाका बारेमा लेख्नुहोस्।\n\n### Context:\nऊर्जा संकट समाधानका लागि सरकारले नवीकरणीय ऊर्जामा लगानी बढाउने नीति लिएको छ। सौर्य ऊर्जा र वायु ऊर्जा उत्पादन गर्ने उद्योगलाई कर छुटको व्यवस्था गरिएको छ। साथै, विद्युत खपत बढाउन विद्युतीय सवारी साधनको प्रयोगलाई प्रोत्साहन गरिनेछ।\n\n### Response:\nनवीकरणीय ऊर्जाको उत्पादन गर्ने उद्योगलाई तर्सँग तालो छन्। सौर्य ऊर्जाको उत्पादन गर्ने उद्योगलाई कर छुटको व्यवस्था गरिएको छ। साथै विद्युतीय उत्पादनको आवास विस्तार गरिने विधान लिइएको छ।'